In [2]:
pip install dictances

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from numpy.linalg import norm
from dictances import bhattacharyya, bhattacharyya_coefficient

In [4]:
from __future__ import division
import json
import numpy as np
from decimal import Decimal
from numpy import expand_dims, array, exp, max
import scipy
from scipy.special import rel_entr
import csv
import statistics
import math
from scipy.spatial import distance
from scipy.stats import wasserstein_distance

In [5]:
# to suppress division errors
np.seterr(divide='ignore', invalid='ignore')
new_final_source_array = []

In [6]:
# Opening final JSON files
contrast_source_data = open('../../../files/lbp/source_domain_lbp.json')

#for target data
contrast_target_data = open('../../../files/lbp/stanford_target_domain_lbp.json')

In [7]:
# returns JSON object
contrast_source_data_d = json.load(contrast_source_data)

contrast_target_data_d = json.load(contrast_target_data)

In [8]:
# kdl dictionary
kdl_dictionary = {}

In [8]:
def KL(p, q):
    a = []
    b = []
    for k in p:
        a.append(Decimal(k))

    for l in q:
        b.append(Decimal(l))

    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)

    return np.sum(np.where(((a != 0) & (b != 0)), a * np.log(a / b), 0))

In [9]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [10]:
# get the data for the contrast target image
contrast_target_data_array = []
for p in range(0, len(contrast_target_data_d['target_images'])):
    for i, j in contrast_target_data_d['target_images'][p].items():
        contrast_target_data_array.append((i,np.array(j)))
contrast_target_data.close()

In [11]:
print(len(contrast_target_data_array))

1568


In [12]:
def hellinger_fast(p, q):
    """Hellinger distance between two discrete distributions.
       In pure Python.
       Fastest version.
    """
    return sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p,q) ])

In [13]:
###contrast comparison code
contrast_All_Images = []
contrast_average_dkl = []

for x in contrast_target_data_array[:4000]:
#for x in target_data_array:    
    image_name_class =  x[0].split('_/') #name[0]; class[1]
    compare_min_max = []
    
    
            
    # get the data for the source images
    for m in range(0, len(contrast_source_data_d['source_images'])):
        #print("source",source_data_d['source_images'][m][0])
        for k, v in contrast_source_data_d['source_images'][m].items():
            # compare the value of v with that of the target image
            # remove the zeros from the list
            new_v = np.array(v).flatten()
          
            #get the top items in x
            items = np.sort(x[1])
            item_size = items[-len(new_v):]
            new_x = item_size.flatten()            
                       
            kdl = rel_entr( new_x, new_v, out=None)
            new_bhattacharyya = bhattacharyya(dict(enumerate(new_x.flatten(), 1)),dict(enumerate(new_v.flatten(), 1)) )
            new_wasserstein = wasserstein_distance(new_x,new_v)
            new_jensen = distance.jensenshannon(new_x,new_v)
            new_hellinger = hellinger_fast(new_x,new_v)
            
            ##compare_min_max.append((image_name_class[0],image_name_class[1],k, max(kdl)))
            compare_min_max.append((image_name_class[0],image_name_class[1],k, max(new_bhattacharyya)))      
    
    ######min_max_v = []
           
    #####for max_kdl in compare_min_max: 
       
        ####min_max_v.append((max_kdl[0],max_kdl[1],max_kdl[2],max_kdl[3]))
    
    ####for df in min_max_v:
        ###max_kdl_v = max(df[3])
        ####min_kdl_v = min(df[3])
        ###if(max_kdl_v >=0 and max_kdl_v <0.5 ):
            ###t_images_btween_0_05.append((df[2],image_name_class[0],image_name_class[1],max_kdl_v,min_kdl_v))
        ###elif(max_kdl_v >=0.5 and max_kdl_v <1 ):
            ###t_images_btween_05_1.append((df[2],image_name_class[0],image_name_class[1],max_kdl_v,min_kdl_v))
            
        ###elif(max_kdl_v >=1 and max_kdl_v <2 ):
            ###t_images_btween_1_2.append((df[2],image_name_class[0],image_name_class[1], max_kdl_v,min_kdl_v))
        ###elif(max_kdl_v >=2 and max_kdl_v <5 ):
            ####t_images_grt_2_5.append((df[2],image_name_class[0],image_name_class[1], max_kdl_v,min_kdl_v))
        ####elif(max_kdl_v >=5 and max_kdl_v <10 ):
            ###t_images_grt_5_10.append((df[2],image_name_class[0],image_name_class[1], max_kdl_v,min_kdl_v))
        ####elif max_kdl_v >=10:
            ###t_images_grt_10.append((df[2],image_name_class[0],image_name_class[1], max_kdl_v,min_kdl_v))
            
    #print(min_max_v)
    #####All_Images.append((image_name_class[0],image_name_class[1],list((len(t_images_btween_0_05),len(t_images_btween_05_1),len(t_images_btween_1_2),len(t_images_grt_2_5), len(t_images_grt_5_10),len(t_images_grt_10)))))
    contrast_All_Images.append(compare_min_max)
contrast_source_data.close()

In [14]:
contrast_sum_dkls = 0
contrast_all_nums = []
for df in contrast_All_Images:
    contrast_sum_dkls = contrast_sum_dkls + df[1][3]
    contrast_all_nums.append(df[1][3])
    #print(df[1][3])
contrast_average_dkl_v = contrast_sum_dkls/ len(contrast_All_Images)
contrast_median_v = statistics.median(contrast_all_nums)
#get the values below and above the average
contrast_below_avrg = []
contrast_above_avrg = []

for fg in contrast_All_Images:
    if fg[1][3] < contrast_average_dkl_v:
        contrast_below_avrg.append(fg[1])
    elif fg[1][3] >= contrast_average_dkl_v:
        contrast_above_avrg.append(fg[1])

In [15]:
contrast_above_other = []
for w in contrast_above_avrg:
    image_n = w[1]+"/"+w[0]
    image_c = w[1]
    
    contrast_above_other.append((image_n,image_c))

In [16]:
print(len(contrast_above_other))

735


In [17]:
#save the image and class in csv
with open('../../../files/stanford/lbp/bhattarchayya_stanford_lbp_above.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)  # write the header
    writer.writerow(["file", "label"])
    for l_item in contrast_above_other:
        writer.writerow(l_item)


        



In [18]:
contrast_below_other = []
for z in contrast_below_avrg:
    image_n = z[1]+"/"+z[0]
    image_c = z[1]
    contrast_below_other.append((image_n,image_c))



In [19]:
print(len(contrast_below_other))

833


In [20]:
#save the image and class in csv
with open('../../../files/stanford/lbp/bhattarchayya_stanford_lbp_below.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)  # write the header
    writer.writerow(["file", "label"])
    for l_item in contrast_below_other:
        writer.writerow(l_item)